<div style="float:right; padding-top: 15px; padding-right: 15px">
    <div>
        <a href="https://whiteboxml.com">
            <img src="https://whiteboxml.com/static/img/logo/black_bg_white.svg" width="250">
        </a>
    </div>
</div>

## 0. python imports & setup

for learning purposes, libraries will be imported inside its corresponding usage section...

## 1. data loading

In [40]:
import pandas as pd

* diamonds: labeled data we can use for training and testing
* diamonds_predict: diamonds to predict its price and upload result to Kaggle

In [41]:
diamonds = pd.read_csv('../data/raw/diamonds_train.csv')
diamonds_predict = pd.read_csv('../data/raw/diamonds_predict.csv')

In [42]:
diamonds.head().T

,0,1,2,3,4
carat,1.21,0.32,0.71,0.41,1.02
cut,Premium,Very Good,Fair,Good,Ideal
color,J,H,G,D,G
clarity,VS2,VS2,VS1,SI1,SI1
depth,62.4,63,65.5,63.8,60.5
table,58,57,55,56,59
price,4268,505,2686,738,4882
x,6.83,4.35,5.62,4.68,6.55
y,6.79,4.38,5.53,4.72,6.51
z,4.25,2.75,3.65,3,3.95


as you can see, there are both categorical and numerical columns...

## 2. eda

this section is up to you! this guided lesson is about a machine learning pipeline...

## 3. ml preprocessing

in this section I will teach how to use scikit-learn's Pipiline and ColumnTransformer, one of the best practices for composing preprocessing and modeling in a single and elegand class... pay attention as it is hard to understand...

In [43]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

* https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html
* https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer
* https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html
* https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

let's identify numerical and categorical features...

In [44]:
NUM_FEATS = ['carat', 'depth', 'table', 'x', 'y', 'z']
CAT_FEATS = ['cut', 'color', 'clarity']
FEATS = NUM_FEATS + CAT_FEATS
TARGET = 'price'

let's define a preprocessing transformer for numerical columns...

In [45]:
numeric_transformer = \
Pipeline(steps=[('imputer', SimpleImputer(strategy='median')), 
                ('scaler', RobustScaler())])

let's define a preprocessing transformer for categorical columns...

In [46]:
categorical_transformer = \
Pipeline(steps=[('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
                ('onehot', OneHotEncoder(handle_unknown='ignore'))])

let's join these transformers using a `ColumnTransformer`:

In [47]:
preprocessor = \
ColumnTransformer(transformers=[('num', numeric_transformer, NUM_FEATS),
                                ('cat', categorical_transformer, CAT_FEATS)])

inspecting the full preprocessor:

In [48]:
preprocessor

ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0.3,
                  transformer_weights=None,
                  transformers=[('num',
                                 Pipeline(memory=None,
                                          steps=[('imputer',
                                                  SimpleImputer(add_indicator=False,
                                                                copy=True,
                                                                fill_value=None,
                                                                missing_values=nan,
                                                                strategy='median',
                                                                verbose=0)),
                                                 ('scaler',
                                                  RobustScaler(copy=True,
                                                               quantile_range=(25.0,
                         

how does this preprocessing looks like?

at least in this case, it is at the cost of interpretability of transformed DataFrame...

In [49]:
pd.DataFrame(data=preprocessor.fit_transform(diamonds)).head()

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,0.796875,0.400000,0.333333,0.622951,0.593407,0.648889,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,-0.593750,0.800000,0.000000,-0.732240,-0.730769,-0.684444,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.015625,2.466667,-0.666667,-0.038251,-0.098901,0.115556,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,-0.453125,1.333333,-0.333333,-0.551913,-0.543956,-0.462222,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.500000,-0.866667,0.666667,0.469945,0.439560,0.382222,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


## 4. train a simple model

first, lets train a simple model using holdout, train - test split...

In [50]:
from sklearn.model_selection import train_test_split

In [51]:
diamonds_train, diamonds_test = train_test_split(diamonds)

In [52]:
print(diamonds_train.shape)
print(diamonds_test.shape)

(30341, 10)
(10114, 10)


let's choose a model from scikit-learn cheatsheet: https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html

In [53]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor

model= Pipeline(steps=[('preprocessor', preprocessor),
                       ('regressor', GradientBoostingRegressor())])

In [54]:
model.fit(diamonds_train[FEATS], diamonds_train[TARGET]);

## 5. check model performance on test and train data

In [55]:
from sklearn.metrics import mean_squared_error

In [56]:
y_test = model.predict(diamonds_test[FEATS])
y_train = model.predict(diamonds_train[FEATS])

In [57]:
print(f"test error: {mean_squared_error(y_pred=y_test, y_true=diamonds_test[TARGET], squared=False)}")
print(f"train error: {mean_squared_error(y_pred=y_train, y_true=diamonds_train[TARGET], squared=False)}")

test error: 759.0928911882203
train error: 703.9727002606021


## 6. check model performance using cross validation

In [58]:
from sklearn.model_selection import cross_val_score

In [59]:
scores = cross_val_score(model, 
                         diamonds[FEATS], 
                         diamonds[TARGET], 
                         scoring='neg_root_mean_squared_error', 
                         cv=5, n_jobs=-1)

In [60]:
import numpy as np
np.mean(-scores)

733.2630956276773

## 7. optimize model using grid search

In [61]:
from sklearn.model_selection import RandomizedSearchCV

In [62]:
param_grid = {
    'preprocessor__num__imputer__strategy': ['mean', 'median'],
    'regressor__n_estimators': [16, 32, 64, 128, 256, 512],
    'regressor__max_depth': [2, 4, 8, 16],
}

grid_search = RandomizedSearchCV(model, 
                                 param_grid, 
                                 cv=5, 
                                 verbose=10, 
                                 scoring='neg_root_mean_squared_error', 
                                 n_jobs=6,
                                 n_iter=32)

grid_search.fit(diamonds[FEATS], diamonds[TARGET])

Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 tasks      | elapsed:    1.1s
[Parallel(n_jobs=6)]: Done   6 tasks      | elapsed:    1.9s
[Parallel(n_jobs=6)]: Done  13 tasks      | elapsed:    7.3s
[Parallel(n_jobs=6)]: Done  20 tasks      | elapsed:   10.8s
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:   16.4s
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:   23.5s
[Parallel(n_jobs=6)]: Done  49 tasks      | elapsed:   42.4s
[Parallel(n_jobs=6)]: Done  60 tasks      | elapsed:   52.7s
[Parallel(n_jobs=6)]: Done  73 tasks      | elapsed:  2.6min
[Parallel(n_jobs=6)]: Done  86 tasks      | elapsed:  3.7min
[Parallel(n_jobs=6)]: Done 101 tasks      | elapsed:  4.2min
[Parallel(n_jobs=6)]: Done 116 tasks      | elapsed:  4.7min
[Parallel(n_jobs=6)]: Done 133 tasks      | elapsed:  5.1min
[Parallel(n_jobs=6)]: Done 160 out of 160 | elapsed:  6.1min finished


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('preprocessor',
                                              ColumnTransformer(n_jobs=None,
                                                                remainder='drop',
                                                                sparse_threshold=0.3,
                                                                transformer_weights=None,
                                                                transformers=[('num',
                                                                               Pipeline(memory=None,
                                                                                        steps=[('imputer',
                                                                                                SimpleImputer(add_indicator=False,
                                                                                     

In [63]:
grid_search.best_params_

{'regressor__n_estimators': 128,
 'regressor__max_depth': 8,
 'preprocessor__num__imputer__strategy': 'median'}

In [64]:
grid_search.best_estimator_

Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  Pipeline(memory=None,
                                                           steps=[('imputer',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value=None,
                                                                                 missing_values=nan,
                                                                                 strategy='median',
                                                             

In [65]:
grid_search.best_score_

-542.5965813523222

## 8. prepare submission

In [66]:
y_pred = grid_search.predict(diamonds_predict[FEATS])

In [67]:
submission_df = pd.DataFrame({'id': diamonds_predict['id'], 'price': y_pred})

In [68]:
submission_df.head()

,id,price
0,0,2907.440999
1,1,5602.065190
2,2,9594.540692
3,3,3984.876501
4,4,1600.108420


In [69]:
submission_df.describe()

,id,price
count,13485.000000,13485.000000
mean,6742.000000,3949.487298
std,3892.928525,3947.761723
min,0.000000,263.061099
25%,3371.000000,936.529484
50%,6742.000000,2446.717212
75%,10113.000000,5323.594204
max,13484.000000,19017.517224


In [70]:
submission_df.price.clip(0, 20000, inplace=True)

In [71]:
submission_df.to_csv('../submissions/diamonds_GBR_15-18.csv', index=False)

## 9. let's try more models...

<div style="padding-top: 25px; float: right">
    <div>    
        <i>&nbsp;&nbsp;© Copyright by</i>
    </div>
    <div>
        <a href="https://whiteboxml.com">
            <img src="https://whiteboxml.com/static/img/logo/black_bg_white.svg" width="125">
        </a>
    </div>
</div>